# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()
print(collections)

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")
# Display the first document in the results using pprint
first_result = establishments.find_one(query)
pprint(first_result)

Number of establishments with a hygiene score of 20: 0
None


In [7]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
df = pd.DataFrame(list(results))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
df.head(10)

Number of rows in the DataFrame: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'address.localAuthority': 'London', 'ratingValue': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with a ratingValue >= 4: {count}")

# Display the first document in the results using pprint
first_result = establishments.find_one(query)
pprint(first_result)

In [ ]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
df = pd.DataFrame(list(results))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074  # example latitude for "Penang Flavours"
longitude = -0.1278  # example longitude for "Penang Flavours"

query = {
    'ratingValue': 5,
    'location': {
        '$geoWithin': {
            '$centerSphere': [[longitude, latitude], degree_search / 3963.2]  # 3963.2 miles per degree
        }
    }
}

# Sort by hygiene score (ascending)
sort = [('scores.Hygiene', 1)]  # Ascending order of hygiene score
limit = 5

# Print the results
results = establishments.find(query).sort(sort).limit(limit)

for result in results:
    pprint(result)


In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(results))
df.head(5)


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {'$match': {'scores.Hygiene': 0}},  # Match establishments with a hygiene score of 0
    {'$group': {'_id': '$address.localAuthority', 'count': {'$sum': 1}}},  # Group by Local Authority
    {'$sort': {'count': -1}}  # Sort by count in descending order
]

# Print the number of documents in the result
results = establishments.aggregate(pipeline)

# Print the first 10 results
for result in results:
    pprint(result)


In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
df.head(10)
